In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv
/kaggle/input/music-data/tested_data_using_split_genre.csv
/kaggle/input/music-data/trained_data_using_split_genre.csv


In [2]:
train = pd.read_csv('/kaggle/input/music-data/trained_data_using_split_genre.csv')
test = pd.read_csv('/kaggle/input/music-data/tested_data_using_split_genre.csv')

In [3]:
train.head()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,...,Genre_0,Genre_1,Genre_2,Genre_3,Genre_4,Genre_5,Genre_6,Genre_7,Genre_8,Genre_9
0,34,98,63.84,74.81,4,3,3.0,0.0,1,31.41998,...,0,0,0,0,0,0,0,0,0,1
1,24,19,119.80,66.95,2,0,3.0,2.0,-1,88.01241,...,0,1,0,0,0,0,0,0,0,0
2,40,8,73.90,69.97,5,1,3.0,0.0,-1,44.92531,...,0,0,1,0,0,0,0,0,0,0
3,10,40,67.17,57.22,1,2,3.0,2.0,1,46.27824,...,0,0,0,0,0,0,0,0,1,0
4,31,85,110.51,80.07,1,0,3.0,3.0,0,75.61031,...,0,0,0,1,0,0,0,0,0,0


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790770 entries, 0 to 790769
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 790770 non-null  int64  
 1   Episode_Title                790770 non-null  int64  
 2   Episode_Length_minutes       790770 non-null  float64
 3   Host_Popularity_percentage   790770 non-null  float64
 4   Publication_Day              790770 non-null  int64  
 5   Publication_Time             790770 non-null  int64  
 6   Guest_Popularity_percentage  790770 non-null  float64
 7   Number_of_Ads                790770 non-null  float64
 8   Episode_Sentiment            790770 non-null  int64  
 9   Listening_Time_minutes       790770 non-null  float64
 10  Episode_Title_num            790770 non-null  int64  
 11  Ad_Density                   790770 non-null  float64
 12  Popularity_Diff              790770 non-null  float64
 13 

In [5]:
train.shape, test.shape

((790770, 26), (250000, 25))

In [6]:
def add_engineered_features(df):
    df['Ad_Effect'] = df['Number_of_Ads'] * df['Episode_Sentiment']
    df['Is_Weekend'] = df['Publication_Day'].isin([5, 6]).astype(int)
    
    def categorize_time(hour):
        if 5 <= hour < 12:
            return 'Morning'
        elif 12 <= hour < 17:
            return 'Afternoon'
        elif 17 <= hour < 21:
            return 'Evening'
        else:
            return 'Night'
    
    df['Time_of_Day_Category'] = df['Publication_Time'].apply(categorize_time)
    df = pd.get_dummies(df, columns=['Time_of_Day_Category'], drop_first=True)
    
    return df


In [7]:
train = add_engineered_features(train)
test = add_engineered_features(test)

In [8]:
train.head()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,...,Genre_2,Genre_3,Genre_4,Genre_5,Genre_6,Genre_7,Genre_8,Genre_9,Ad_Effect,Is_Weekend
0,34,98,63.84,74.81,4,3,3.0,0.0,1,31.41998,...,0,0,0,0,0,0,0,1,0.0,0
1,24,19,119.80,66.95,2,0,3.0,2.0,-1,88.01241,...,0,0,0,0,0,0,0,0,-2.0,0
2,40,8,73.90,69.97,5,1,3.0,0.0,-1,44.92531,...,1,0,0,0,0,0,0,0,-0.0,1
3,10,40,67.17,57.22,1,2,3.0,2.0,1,46.27824,...,0,0,0,0,0,0,1,0,2.0,0
4,31,85,110.51,80.07,1,0,3.0,3.0,0,75.61031,...,0,1,0,0,0,0,0,0,0.0,0


In [9]:
test.head()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Episode_Title_num,...,Genre_2,Genre_3,Genre_4,Genre_5,Genre_6,Genre_7,Genre_8,Genre_9,Ad_Effect,Is_Weekend
0,11,71,78.96,38.11,2,1,3.0,1.0,0,73,...,1,0,0,0,0,0,0,0,0.0,0
1,36,16,27.87,71.29,3,2,3.0,0.0,0,23,...,0,0,0,1,0,0,0,0,0.0,0
2,24,3,69.10,67.89,0,1,3.0,0.0,1,11,...,0,0,0,0,0,0,0,0,0.0,0
3,4,71,115.39,23.40,3,2,3.0,2.0,1,73,...,0,0,0,0,0,0,0,0,2.0,0
4,27,46,72.32,58.10,6,2,3.0,2.0,0,50,...,0,0,1,0,0,0,0,0,0.0,1


In [10]:
def add_popularity_features(df):
    df['Popularity_Diff_Normalized'] = df['Popularity_Diff'] / (df['Popularity_Average'] + 1)
    df['Popularity_Balance'] = 1 - abs(df['Host_Popularity_percentage'] - df['Guest_Popularity_percentage']) / 100
    return df


In [11]:
train = add_popularity_features(train)
test = add_popularity_features(test)

In [12]:
train.shape, test.shape

((790770, 30), (250000, 29))

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790770 entries, 0 to 790769
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Podcast_Name                 790770 non-null  int64  
 1   Episode_Title                790770 non-null  int64  
 2   Episode_Length_minutes       790770 non-null  float64
 3   Host_Popularity_percentage   790770 non-null  float64
 4   Publication_Day              790770 non-null  int64  
 5   Publication_Time             790770 non-null  int64  
 6   Guest_Popularity_percentage  790770 non-null  float64
 7   Number_of_Ads                790770 non-null  float64
 8   Episode_Sentiment            790770 non-null  int64  
 9   Listening_Time_minutes       790770 non-null  float64
 10  Episode_Title_num            790770 non-null  int64  
 11  Ad_Density                   790770 non-null  float64
 12  Popularity_Diff              790770 non-null  float64
 13 

In [14]:
train.describe()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,...,Genre_4,Genre_5,Genre_6,Genre_7,Genre_8,Genre_9,Ad_Effect,Is_Weekend,Popularity_Diff_Normalized,Popularity_Balance
count,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,...,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000
mean,23.512608,50.229309,64.344096,59.880366,2.965325,1.521992,2.971209,1.360056,-0.002016,45.444340,...,0.109712,0.083648,0.085474,0.116044,0.114233,0.112769,-0.020105,0.275383,1.715391,0.430908
std,14.122942,28.231235,31.031891,22.889812,1.997226,1.119393,0.235448,1.150412,0.815497,27.137628,...,0.312531,0.276859,0.279585,0.320278,0.318094,0.316310,1.427770,0.446708,0.121405,0.228907
min,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,0.000000,-0.539683,-0.164600
25%,11.000000,26.000000,39.340000,39.450000,1.000000,1.000000,3.000000,0.000000,-1.000000,23.186570,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.641898,0.234100
50%,23.000000,51.000000,63.840000,60.060000,3.000000,1.000000,3.000000,1.000000,0.000000,43.393291,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.755314,0.429100
75%,37.000000,74.000000,90.240000,79.560000,5.000000,3.000000,3.000000,2.000000,1.000000,64.814290,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.811853,0.635300
max,47.000000,99.000000,325.240000,119.460000,6.000000,3.000000,3.000000,103.910000,1.000000,119.970000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,53.370000,1.000000,1.960310,0.999500


## Clipping the target variable

In [16]:
# Calculate clipping thresholds from training data only
lower = train['Listening_Time_minutes'].quantile(0.01)
upper = train['Listening_Time_minutes'].quantile(0.99)

# Apply clipping to train set only
train['Listening_Time_minutes'] = train['Listening_Time_minutes'].clip(lower=lower, upper=upper)


In [18]:
train.describe()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,...,Genre_4,Genre_5,Genre_6,Genre_7,Genre_8,Genre_9,Ad_Effect,Is_Weekend,Popularity_Diff_Normalized,Popularity_Balance
count,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,...,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000,790770.000000
mean,23.512608,50.229309,64.344096,59.880366,2.965325,1.521992,2.971209,1.360056,-0.002016,45.401486,...,0.109712,0.083648,0.085474,0.116044,0.114233,0.112769,-0.020105,0.275383,1.715391,0.430908
std,14.122942,28.231235,31.031891,22.889812,1.997226,1.119393,0.235448,1.150412,0.815497,27.031441,...,0.312531,0.276859,0.279585,0.320278,0.318094,0.316310,1.427770,0.446708,0.121405,0.228907
min,0.000000,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,0.000000,-0.539683,-0.164600
25%,11.000000,26.000000,39.340000,39.450000,1.000000,1.000000,3.000000,0.000000,-1.000000,23.186570,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.641898,0.234100
50%,23.000000,51.000000,63.840000,60.060000,3.000000,1.000000,3.000000,1.000000,0.000000,43.393291,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.755314,0.429100
75%,37.000000,74.000000,90.240000,79.560000,5.000000,3.000000,3.000000,2.000000,1.000000,64.814290,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.811853,0.635300
max,47.000000,99.000000,325.240000,119.460000,6.000000,3.000000,3.000000,103.910000,1.000000,109.388870,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,53.370000,1.000000,1.960310,0.999500


In [19]:
X_train = train.drop(columns=['Listening_Time_minutes'])
y_train = train['Listening_Time_minutes']

# Test features (X) for test data (no target variable in the test set)
X_test = test

## Using RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize model
model = RandomForestRegressor()

# Train the model
model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation on training data
cv_scores = cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
print(f"Average CV RMSE: {-cv_scores.mean()}")
